# <B> Clean up </B>

## AutoReload

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## clean

In [21]:
import os
import boto3
from utils.ssm import parameter_store

In [22]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

* S3 Bucket

In [23]:
strBucketName = pm.get_params(key=strPrefix + "BUCKET")

In [24]:
strQuery = "".join(["aws s3 rm --recursive ", '"s3://', strBucketName, '"'])
strResponse = os.popen(strQuery).read()
strQuery = "".join(["aws s3 rb ", '"s3://', strBucketName, '"'])
strResponse = os.popen(strQuery).read()

* endpoint

In [25]:
class clean_up():
    
    def __init__(self, ):    
        pass
    
    def delete_endpoint(self, client, endpoint_name ,is_del_model=True):
        
        response = client.describe_endpoint(EndpointName=endpoint_name)
        EndpointConfigName = response['EndpointConfigName']

        response = client.describe_endpoint_config(EndpointConfigName=EndpointConfigName)
        model_name = response['ProductionVariants'][0]['ModelName']    

        if is_del_model: # 모델도 삭제 여부 임.
            client.delete_model(ModelName=model_name)    

        client.delete_endpoint(EndpointName=endpoint_name)
        client.delete_endpoint_config(EndpointConfigName=EndpointConfigName)    

        print(f'--- Deleted model: {model_name}')
        print(f'--- Deleted endpoint: {endpoint_name}')
        print(f'--- Deleted endpoint_config: {EndpointConfigName}')  

In [26]:
clean = clean_up()
sm_client = boto3.client('sagemaker')
strEndPointName = pm.get_params(key=strPrefix + "ENDPOINT-NAME")
clean.delete_endpoint(sm_client, strEndPointName ,is_del_model=True)

--- Deleted model: pytorch-inference-2023-02-16-01-02-18-894
--- Deleted endpoint: endpoint-SM-BYOM-1676509337
--- Deleted endpoint_config: endpoint-SM-BYOM-1676509337


* parameter store

In [27]:
listParams = pm.get_all_params()
listParams = [strParam for strParam in listParams if strPrefix in strParam] + ["PREFIX"]
pm.delete_param(listParams)

  parameters: ['SM-BYOM-ACCOUNT-ID', 'SM-BYOM-BUCKET', 'SM-BYOM-DATA-PATH', 'SM-BYOM-ENDPOINT-NAME', 'SM-BYOM-REGION', 'SM-BYOM-S3-MODEL-ARTIFACT', 'SM-BYOM-SAGEMAKER-ROLE-ARN', 'SM-BYOM-TRAIN-IMAGE-URI', 'PREFIX'] is deleted successfully
